# One Hot Simple

This is a very simple version of the model. One LSTM. Converts all the training to one hot representation before feeding it in, which means it can only train on a very small fraction of the full training data. As a result, I need to move to something more complex, using fit_generator() instead of fit().

Anyway, I've learnt a lot from this, still.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

fs_path = '/content/drive/My Drive/Study/Machine Learning/firststeps/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import json
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

Using TensorFlow backend.


Set up local :

In [0]:
with open(fs_path + 'training_data/training_data100.json', 'r') as f:
    X_events, Y_events, Tx = json.load(f)
    
print('training data collected')

m = len(X_events)
print('m: ', m)
Ty = Tx



training data collected
m:  860


We use the following function to make the data one hot:

In [0]:
def make_one_hot(examples, n_values=333):
    """ takes in a list of training examples, returns them in one hot format
    
    Inputs
    ----------
    examples : list
        should contain training examples, which themselves are lists of events expressed in integers
    
    """
    arr = np.empty((len(examples), len(examples[0]), n_values), dtype=object)
    for i in range(len(examples)):
        one_hots = to_categorical(examples[i], num_classes=n_values, dtype='float32')
        arr[i] = one_hots
    return arr
  
X = make_one_hot(X_events)
Y = make_one_hot(Y_events)

print("X dimensions: ", X.shape)
print("Type of X: ", type(X))
print("Type of X[0]: ", type(X[0]))
print("Type of X[0][0]: ", type(X[0][0]))

X dimensions:  (860, 256, 333)
Type of X:  <class 'numpy.ndarray'>
Type of X[0]:  <class 'numpy.ndarray'>
Type of X[0][0]:  <class 'numpy.ndarray'>


Set up the variables for the model, and the layers

In [0]:
#number of hidden LSTM states
n_a = 256

# no of unique events
n_values = 333

# # some of the objects for our network
reshapor = Reshape((1, n_values))
LSTM_cell = LSTM(n_a, return_state = True)
densor = Dense(n_values, activation='softmax')

W0714 09:31:46.021742 140562802812800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Here is the model:

In [0]:
def musicmodel(Tx, n_a, n_values):
    """
    Implement the model
    
    Arguments:
    Tx -- length of the sequence in a corpus
    n_a -- the number of activations used in our model
    n_values -- number of unique values in the music data 
    
    Returns:
    model -- a keras model with the 
    """
    
    # Define the input of your model with a shape 
    X = Input(shape=(Tx, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0

    # Step 1: Create empty list to append the outputs while you iterate (≈1 line)
    outputs = []
    
    # Step 2: Loop
    for t in range(Tx):
        
        # Step 2.A: select the "t"th time step vector from X. 
        x = Lambda(lambda x: x[:,t,:])(X)
        # Step 2.B: Use reshapor to reshape x to be (1, n_values) (≈1 line)
        x = reshapor(x)
        # Step 2.C: Perform one step of the LSTM_cell
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        # Step 2.D: Apply densor to the hidden state output of LSTM_Cell
        out = densor(a)
        # Step 2.E: add the output to "outputs"
        outputs.append(out)
        
    # Step 3: Create model instance
    model = Model(inputs=[X,a0,c0],outputs=outputs)
    
    ### END CODE HERE ###
    
    return model

To run the model:

In [0]:
model = musicmodel(Tx, n_a, n_values)


opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

# model.load_weights('weights/model1layer256lstm400epochs.h5')
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

a0 = np.zeros((m, n_a))
c0 = np.zeros((m, n_a))


Y = Y.transpose(1,0,2)
print("Y dimensions: ", Y.shape)


checkpoint = ModelCheckpoint(fs_path + "weightsCOLAB.256.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
callbacks_list = [checkpoint]

history = model.fit([X, a0, c0], list(Y), validation_split=0.2, epochs=400, callbacks=callbacks_list)

model.save_weights(fs_path + 'weights/COLABmodel1layer256lstm404epochs.h5')


# summarize history for loss, from https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# model.load_weights('weights/model1layer256lstm60epochs.h5)

W0714 09:32:02.609440 140562802812800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 09:32:02.638581 140562802812800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0714 09:32:36.814003 140562802812800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0714 09:32:37.092019 140562802812800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



Y dimensions:  (256, 860, 333)


W0714 09:32:46.588183 140562802812800 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0714 09:35:17.640023 140562802812800 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 688 samples, validate on 172 samples
Epoch 1/400
688/688 [==============================] - 228s 331ms/step - loss: 1282.8419 - dense_1_loss: 5.0860 - dense_1_acc: 0.0044 - dense_1_acc_1: 0.0465 - dense_1_acc_2: 0.0029 - dense_1_acc_3: 0.0422 - dense_1_acc_4: 0.0712 - dense_1_acc_5: 0.0305 - dense_1_acc_6: 0.0538 - dense_1_acc_7: 0.0698 - dense_1_acc_8: 0.0596 - dense_1_acc_9: 0.0669 - dense_1_acc_10: 0.0610 - dense_1_acc_11: 0.0785 - dense_1_acc_12: 0.0640 - dense_1_acc_13: 0.0581 - dense_1_acc_14: 0.0669 - dense_1_acc_15: 0.0581 - dense_1_acc_16: 0.0640 - dense_1_acc_17: 0.0552 - dense_1_acc_18: 0.0669 - dense_1_acc_19: 0.0712 - dense_1_acc_20: 0.0712 - dense_1_acc_21: 0.0465 - dense_1_acc_22: 0.0770 - dense_1_acc_23: 0.0552 - dense_1_acc_24: 0.0480 - dense_1_acc_25: 0.0770 - dense_1_acc_26: 0.0654 - dense_1_acc_27: 0.0698 - dense_1_acc_28: 0.0610 - dense_1_acc_29: 0.0465 - dense_1_acc_30: 0.0596 - dense_1_acc_31: 0.0523 - dense_1_acc_32: 0.0509 - dense_1_acc_33: 0.0683 - de

Here is the inference model:

In [0]:
def music_inference_model(LSTM_cell, densor, n_values, n_a, Ty = 100):
    """
    Uses the trained "LSTM_cell" and "densor" from model() to generate a sequence of values.
    
    Arguments:
    LSTM_cell -- the trained "LSTM_cell" from model(), Keras layer object
    densor -- the trained "densor" from model(), Keras layer object
    n_values -- integer, umber of unique values
    n_a -- number of units in the LSTM_cell
    Ty -- integer, number of time steps to generate
    
    Returns:
    inference_model -- Keras model instance
    """
    
    # Define the input of your model with a shape 
    x0 = Input(shape=(1, n_values))
    
    # Define s0, initial hidden state for the decoder LSTM
    a0 = Input(shape=(n_a,), name='a0')
    c0 = Input(shape=(n_a,), name='c0')
    a = a0
    c = c0
    x = x0

    ### START CODE HERE ###
    # Step 1: Create an empty list of "outputs" to later store your predicted values (≈1 line)
    outputs = []
    
    # Step 2: Loop over Ty and generate a value at every time step
    for t in range(Ty):
        
        # Step 2.A: Perform one step of LSTM_cell (≈1 line)
        a, _, c = LSTM_cell(x, initial_state=[a, c])
        
        # Step 2.B: Apply Dense layer to the hidden state output of the LSTM_cell (≈1 line)
        out = densor(a)

        # Step 2.C: Append the prediction "out" to "outputs". out.shape = (None, 78) (≈1 line)
        outputs.append(out)
        
        # Step 2.D: Select the next value according to "out", and set "x" to be the one-hot representation of the
        #           selected value, which will be passed as the input to LSTM_cell on the next step. We have provided 
        #           the line of code you need to do this. 
        
        x = Lambda(get_max_pred)(x)
        # x = tf.one_hot(np.argmax(out), 333)
        # x = tf.expand_dims(x, axis=-1)
        # x = tf.expand_dims(x, axis=-1)
        
    # Step 3: Create model instance with the correct "inputs" and "outputs" (≈1 line)
    inference_model = Model(inputs=[x0,a0,c0],outputs=outputs)
    
    ### END CODE HERE ###
    
    return inference_model
inference_model = music_inference_model(LSTM_cell, densor, n_values, n_a, Ty)

x_initializer = np.zeros((1, 1, 333))
x_initializer[0][0][308] = 1
a_initializer = np.zeros((1, n_a))
c_initializer = np.zeros((1, n_a))

To make predictions:

In [0]:
def predict_and_sample(inference_model, x_initializer = x_initializer, a_initializer = a_initializer, 
                       c_initializer = c_initializer):
    """
    Predicts the next value of values using the inference model.
    
    Arguments:
    inference_model -- Keras model instance for inference time
    x_initializer -- numpy array of shape (1, 1, 78), one-hot vector initializing the values generation
    a_initializer -- numpy array of shape (1, n_a), initializing the hidden state of the LSTM_cell
    c_initializer -- numpy array of shape (1, n_a), initializing the cell state of the LSTM_cel
    
    Returns:
    results -- numpy-array of shape (Ty, 256), matrix of one-hot vectors representing the values generated
    indices -- numpy-array of shape (Ty, 1), matrix of indices representing the values generated
    """
    
    ### START CODE HERE ###
    # Step 1: Use your inference model to predict an output sequence given x_initializer, a_initializer and c_initializer.
    pred = inference_model.predict([x_initializer,a_initializer,c_initializer])
    # Step 2: Convert "pred" into an np.array() of indices with the maximum probabilities
    #pred is a list of numpy arrays, with probabilities of events at every time step
    indices = np.argmax(pred, axis=-1)
    # Step 3: Convert indices to one-hot vectors, the shape of the results should be (Ty, n_values)
    results = to_categorical(indices, num_classes=n_values)
    ### END CODE HERE ###
    return results, indices

results, indices = predict_and_sample(inference_model, x_initializer, a_initializer, c_initializer)
print("np.argmax(results[12]) =", np.argmax(results[12]))
print("np.argmax(results[17]) =", np.argmax(results[17]))
print("list(indices[12:18]) =", list(indices[12:18]))

dump_pickle_data((results, indices), '4_epochs_try.pkl')

print(callbacks_list)